In [4]:
from keras import backend as K
from keras.models import model_from_json
import tensorflow.compat.v1 as tf

with tf.Graph().as_default():
  with tf.Session() as sess:
    path=""
    path=input("Enter the path where you extracted aimage folder ending with / = ")
    json_file = open(path+'aimage/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(path+'aimage/data_weights.h5')
    model.summary()
    print(model.outputs)
    print(model.inputs)
    init = tf.global_variables_initializer()
    sess.run(init)
    saver = tf.train.Saver()
    saver.save(sess,path+'tensorflowModel.ckpt')
    for node in sess.graph.as_graph_def().node:
        node.device=""
    tf.train.write_graph(sess.graph.as_graph_def(), path, 'tensorflowModel.pb', as_text=True)
    freeze_graph=tf.graph_util.convert_variables_to_constants(sess,sess.graph.as_graph_def(),[out.op.name for out in model.outputs])
    tf.io.write_graph(freeze_graph,path,'tensorflowFrozenModel.pb', as_text=False)
    writer = tf.summary.FileWriter(path, sess.graph)


Enter the path where you extracted aimage folder ending with / = E:\AI\
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 61, 61, 64)        4160      
_________________________________________________________________
activation_1 (Activation)    (None, 61, 61, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 32)        32800     
_________________________________________________________________
activation_2 (Activation)    (None, 27, 27, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
________________________________________________

In [ ]:
tensorboard --logdir E:/AI/

In [4]:
import tensorflow.compat.v1 as tf
from tensorflow.python.tools import freeze_graph, optimize_for_inference_lib
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [5]:
import tensorflow.compat.v1 as tf
from keras.models import model_from_json
with tf.Graph().as_default():
  with tf.Session() as sess:
    path=""
    path=input("Enter the path where you extracted aimage folder ending with / = ")
    json_file = open(path+'aimage/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(path+'aimage/data_weights.h5')
    model.summary()
    print(model.outputs)
    print(model.inputs)
    frozen_graph = freeze_session(sess,output_names=[out.op.name for out in model.outputs])
    tf.train.write_graph(frozen_graph, path, "tensorflow_frozen_model.pb", as_text=False)
    frozen_graph_optimized= optimize_for_inference_lib.optimize_for_inference(
            frozen_graph, [out.op.name for out in model.inputs], [out.op.name for out in model.outputs],
            tf.float32.as_datatype_enum)
    with tf.gfile.GFile(path + 'tensorflow_frozen_model_serialized' + '.pb', "wb") as f:
        f.write(frozen_graph_optimized.SerializeToString())

Enter the path where you extracted aimage folder ending with / = E:\AI\
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 61, 61, 64)        4160      
_________________________________________________________________
activation_1 (Activation)    (None, 61, 61, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 32)        32800     
_________________________________________________________________
activation_2 (Activation)    (None, 27, 27, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
________________________________________________